## SPY 1995-2024 Historical Data Feature Expansion

This script performs leakage-safe feature engineering on historical SPY daily OHLCV data to prepare it for supervised machine learning. It constructs a set of simple but informative predictors using only information available at the end of each trading day, including daily returns, lagged returns, rolling volatility, moving averages, momentum measures, volume dynamics, and calendar effects. The script also defines the prediction target as the direction of the next day’s price movement, ensuring that no future information is used in the input features. The resulting dataset is chronologically ordered and suitable for time-series classification experiments evaluating next-day market direction.

In [6]:
#!/usr/bin/env python3
"""
Leakage-safe feature engineering for SPY OHLCV time series.

Creates:
- Daily return ret_1d (Close_t / Close_{t-1} - 1)
- Lagged returns ret_1d_lag1..ret_1d_lagK
- Rolling volatility of returns (std) over multiple windows
- Moving averages of Close and price-to-MA ratios (trend/level)
- Momentum features (k-day return using Close_t and Close_{t-k})
- Volume changes (pct change), rolling volume means, volume z-score vs rolling window
- Calendar effects: day-of-week, month, day-of-month, week-of-year (optional)
- Label y_next_dir = sign(Close_{t+1} - Close_t)

Input:  SPY_1995_2024_raw_data.csv
Output: SPY_1995_2024_features_leakfree_v2.csv
"""

from __future__ import annotations

import numpy as np
import pandas as pd


INPUT_CSV = "SPY_1995_2024_raw_data.csv"
OUTPUT_CSV = "SPY_1995_2024_features_leakfree_v2.csv"


# ---- Feature config (keep simple but informative) ----
LAG_K = 10                        # lagged daily returns up to t-k
RET_VOL_WINDOWS = (5, 10, 20)     # rolling vol windows on returns
MA_WINDOWS = (5, 10, 20, 50)      # moving averages on Close
MOM_WINDOWS = (5, 10, 20)         # momentum windows (k-day return)
VOL_ROLL_WINDOWS = (5, 20)        # rolling mean/std windows on Volume


def _pick_close_column(df: pd.DataFrame) -> str:
    candidates = ["Close", "close", "Adj Close", "Adj_Close", "adj close", "adj_close"]
    for c in candidates:
        if c in df.columns:
            return c
    raise ValueError(
        f"Could not find a close column. Expected one of: {candidates}. "
        f"Found columns: {list(df.columns)}"
    )


def _pick_date_column(df: pd.DataFrame) -> str:
    candidates = ["Date", "date", "timestamp", "Datetime", "datetime", "Time", "time"]
    for c in candidates:
        if c in df.columns:
            return c
    raise ValueError(
        f"Could not find a date column. Expected one of: {candidates}. "
        f"Found columns: {list(df.columns)}"
    )


def main() -> None:
    df = pd.read_csv(INPUT_CSV)

    # Parse date + sort chronologically
    date_col = _pick_date_column(df)
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df.dropna(subset=[date_col]).sort_values(date_col).reset_index(drop=True)

    close_col = _pick_close_column(df)

    # Coerce core numeric columns (best-effort)
    for col in ["Open", "High", "Low", close_col, "Volume"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # ------------------------------------------------------------------
    # 1) Calendar features (all known from Date at time t; leak-free)
    # ------------------------------------------------------------------
    df["dow"] = df[date_col].dt.dayofweek          # 0=Mon ... 6=Sun (trading days are 0-4)
    df["month"] = df[date_col].dt.month            # 1..12
    df["dom"] = df[date_col].dt.day                # day of month 1..31
    # ISO week can be helpful; keep as int
    df["weekofyear"] = df[date_col].dt.isocalendar().week.astype(int)

    # ------------------------------------------------------------------
    # 2) Returns and lagged returns (leak-free: uses <= t)
    # ------------------------------------------------------------------
    df["ret_1d"] = df[close_col].pct_change()  # (Close_t / Close_{t-1}) - 1

    # Lagged returns ret_1d_lag1..ret_1d_lagK
    for k in range(1, LAG_K + 1):
        df[f"ret_1d_lag{k}"] = df["ret_1d"].shift(k)

    # ------------------------------------------------------------------
    # 3) Rolling volatility of returns (std) (leak-free: window ends at t)
    # ------------------------------------------------------------------
    for w in RET_VOL_WINDOWS:
        df[f"ret_vol_{w}d"] = df["ret_1d"].rolling(window=w, min_periods=w).std()

    # ------------------------------------------------------------------
    # 4) Moving averages of price + price-to-MA ratios (trend/level)
    #    (leak-free: uses Close up to t)
    # ------------------------------------------------------------------
    for w in MA_WINDOWS:
        ma = df[close_col].rolling(window=w, min_periods=w).mean()
        df[f"ma_close_{w}d"] = ma
        # Ratio and distance to MA (both common, simple signals)
        df[f"close_over_ma_{w}d"] = df[close_col] / ma
        df[f"close_minus_ma_{w}d"] = df[close_col] - ma

    # ------------------------------------------------------------------
    # 5) Momentum features: k-day return Close_t vs Close_{t-k}
    #    (leak-free by definition)
    # ------------------------------------------------------------------
    for k in MOM_WINDOWS:
        df[f"mom_{k}d"] = df[close_col] / df[close_col].shift(k) - 1.0

    # ------------------------------------------------------------------
    # 6) Volume dynamics (leak-free: uses Volume up to t)
    # ------------------------------------------------------------------
    if "Volume" in df.columns:
        df["vol_chg_1d"] = df["Volume"].pct_change()

        for w in VOL_ROLL_WINDOWS:
            vmean = df["Volume"].rolling(window=w, min_periods=w).mean()
            vstd = df["Volume"].rolling(window=w, min_periods=w).std()
            df[f"vol_mean_{w}d"] = vmean
            df[f"vol_z_{w}d"] = (df["Volume"] - vmean) / vstd
    else:
        # Keep columns consistent if Volume not present
        df["vol_chg_1d"] = np.nan
        for w in VOL_ROLL_WINDOWS:
            df[f"vol_mean_{w}d"] = np.nan
            df[f"vol_z_{w}d"] = np.nan

    # ------------------------------------------------------------------
    # 7) Label: next-day direction (allowed to use t+1 ONLY for y)
    # ------------------------------------------------------------------
    next_close = df[close_col].shift(-1)
    df["y_next_dir"] = np.sign(next_close - df[close_col]).astype("float")  # -1, 0, +1

    # Optional binary version (uncomment if needed):
    # df["y_next_up"] = ((next_close - df[close_col]) > 0).astype(int)

    # ------------------------------------------------------------------
    # 8) Build final output, dropping unusable rows (NaNs from lag/rolling + last label)
    # ------------------------------------------------------------------
    raw_cols = [c for c in [date_col, "Open", "High", "Low", close_col, "Volume"] if c in df.columns]

    engineered_cols = []
    engineered_cols += ["dow", "month", "dom", "weekofyear"]
    engineered_cols += ["ret_1d"] + [f"ret_1d_lag{k}" for k in range(1, LAG_K + 1)]
    engineered_cols += [f"ret_vol_{w}d" for w in RET_VOL_WINDOWS]
    engineered_cols += [f"ma_close_{w}d" for w in MA_WINDOWS]
    engineered_cols += [f"close_over_ma_{w}d" for w in MA_WINDOWS]
    engineered_cols += [f"close_minus_ma_{w}d" for w in MA_WINDOWS]
    engineered_cols += [f"mom_{k}d" for k in MOM_WINDOWS]
    engineered_cols += ["vol_chg_1d"] + [f"vol_mean_{w}d" for w in VOL_ROLL_WINDOWS] + [f"vol_z_{w}d" for w in VOL_ROLL_WINDOWS]

    label_col = "y_next_dir"

    out_cols = raw_cols + engineered_cols + [label_col]
    out = df[out_cols].copy()

    # Require label + core return and at least the maximum lag/rolling windows.
    # This ensures no NaNs in your chosen feature set.
    required = ["ret_1d", f"ret_1d_lag{LAG_K}", f"ret_vol_{max(RET_VOL_WINDOWS)}d", f"ma_close_{max(MA_WINDOWS)}d", f"mom_{max(MOM_WINDOWS)}d", label_col]
    # Also require volume features only if Volume exists
    if "Volume" in df.columns:
        required += [f"vol_mean_{max(VOL_ROLL_WINDOWS)}d", f"vol_z_{max(VOL_ROLL_WINDOWS)}d"]

    out = out.dropna(subset=required)

    out.to_csv(OUTPUT_CSV, index=False)
    print(f"Saved leak-free dataset to: {OUTPUT_CSV}")
    print(f"Rows: {len(out):,}  Columns: {len(out.columns)}")
    print("Label distribution (y_next_dir):")
    print(out[label_col].value_counts(dropna=False).sort_index())


if __name__ == "__main__":
    main()

Saved leak-free dataset to: SPY_1995_2024_features_leakfree_v2.csv
Rows: 7,751  Columns: 45
Label distribution (y_next_dir):
y_next_dir
-1.0    3539
 0.0      37
 1.0    4175
Name: count, dtype: int64
